# Tutorial 2

## Questions from the tutorial earlier (please answer here)

**Question:** Are there Ns inside the 20bp regions flanking the Ns? If so, why do you think so?

**Answer:**

**Question:** After viewing the normal PAR regions, was your hypothesis correct?

**Answer:**

We'll now look briefly at how STAR and kallisto perform against one another, using some of the tools we learned in the last tutorial

## Load required packages

In [ ]:
# Just as we did last week
import os
import re
import mygene
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from adjustText import adjust_text

In [ ]:
# Set the present working directory with the getcwd() function from the os library
pwd = os.getcwd()

In [ ]:
# Check this is set correctly to '/home/<SUNetID>/BIOC281/Classes/2'
pwd

## Data ingest

In [ ]:
# Read in tab separated values table from STAR using the read_csv function from pandas (pd)
#
# The index_col=0 parameter tells pandas to use the first column as the rownames
# The header=0 parameter tells pandas the first row is real data, not the columns
# which we instead supply with the names parameter.
#
# The join() command from the path section of the os library allows us to construct file paths
# In this case it will return '/home/<SUNetID>/BIOC281/Classes/2/STAResults/male_1p/male.1p.ReadsPerGene.out.tab'
male_STAR = pd.read_csv(os.path.join(pwd, 'STAResults', 'male_1p', 'male.1p.ReadsPerGene.out.tab'), sep="\t", header=0, names=['feature', 'counts', 'pos', 'neg'], index_col=0)

# Read in tab separated values table from kallisto using the read_csv function from pandas (pd)
male_kallisto = pd.read_csv(os.path.join(pwd, 'kallistoResults', 'output', 'abundance.tsv'), sep="\t", index_col=0)

In [ ]:
# Print the STAR data frame
#
# Note how STAR added some metadata, including the number of counts that mapped to multiple loci
# By default, STAR does not include these in the gene counts table
# STAR also reports whether counts came from the positive or negative strand
# Also note there are only 6,018 genes because our minigenome only included chromosomes 1, 2, X, Y, and mitochrondrial
male_STAR

In [ ]:
# Print the kallisto data frame
#
# Note how kallisto has returned RefSeq transcript IDs rather than gene names
# It includes transctipts across the entire genome, which is why there are 86,486 features
# kallisto also supplies the actual and effective (detected) gene lengths (in bases)
# that is used to estimate transcripts per million (tpm)
# The est_counts column is the most directly comparable to STARs output
male_kallisto

In [ ]:
# The RefSeq IDs include a few duplicate entries that have underscores appended onto them (e.g. NR_024540_2)
# We use regular expressions to match and remove them (with the sub() function from the re library)
#
# In this case, the "^" symbol matches the start of each ID and the "$" symbol matches the end
# The two "[^_]+" match 1 or more characters that do not include an "_"
# The parentheses around them allowing us to reference them later
# The "_.*" matches any number of characters following a second "_"
#
# We then replace the matched sequence with the text that comes before the first "_" and
# then the text that comes after the first "_", but before the second "_"
#
# The renames RefSeq IDs are saved into a new column
male_kallisto['collapse_id'] = [re.sub('^([^_]+)_([^_]+)_.*$', '\\1_\\2', x) for x in male_kallisto.index.to_list()]

# Use pandas' groupby() and sum() functions to collapse counts with the same RefSeq ID to a single row
male_kallisto = male_kallisto.groupby('collapse_id').sum()

In [ ]:
# Print the updated table, notice how the number of rows has changed
male_kallisto

## Convert RefSeq IDs to gene names

In [ ]:
# Connect to RefSeq using the mygene library (see https://mygene.info for more uses)
mg = mygene.MyGeneInfo()

# Create a table of RefSeq IDs and corresponding gene names with the querymany() function from mygene
# You can obtain the rownames from a pandas data frame with the index function
# and then convert them to a list (expected by mygene) with the tolist() function
refIDToGene = mg.querymany(male_kallisto.index.tolist(),
             scopes='refseq,symbol',
             species='human',
             as_dataframe=True)

In [ ]:
# Print the table
# Notice how some entries (e.g. the ERCCs) did not have a matching gene symbol and were returned as NaNs)
refIDToGene

In [ ]:
# Create a new column in the kallisto table that has the gene symbols for each RefSeq ID
# We again use index to obtain the rownames for the table
male_kallisto['symbol'] = refIDToGene.symbol[male_kallisto.index].to_list()

In [ ]:
# Find rows without a gene symbol (those with NaNs) and set them to their current rowname
tmp = male_kallisto.symbol.copy()
tmp[tmp.isna()] = tmp[tmp.isna()].index.to_list()
male_kallisto.symbol = tmp.copy()

In [ ]:
# Use pandas' groupby() and sum() functions to collapse counts with the same gene name to a single row
male_kallisto = male_kallisto.groupby('symbol').sum()

In [ ]:
# Print the new kallisto table
# Notice how the number of rows has fallen to 28,402 (in line with the number of human genes)
male_kallisto

## Merge the STAR and kallisto data and compare

In [ ]:
# Identify the genes the STAR and kallisto data have in common (those on the minigenome)
tmp = male_STAR.index.intersection(male_kallisto.index)

# Subset both tables to only include those genes with the loc[] command from pandas
male_kallisto_common = male_kallisto.loc[tmp]
male_STAR_common = male_STAR.loc[tmp]

**Question:** After subsetting the kallisto table, are the transcript per millions values still OK to use?

**Answer:**

In [ ]:
# Create a data frame that contains the STAR and kallisto absolute and log normalized counts as well as their absolute difference
tmp = pd.DataFrame()
tmp['STAR'] = male_STAR_common.counts
tmp['Kallisto'] = male_kallisto_common.est_counts
tmp['log_STAR'] = np.log10(tmp.STAR + 1)
tmp['log_Kallisto'] = np.log10(tmp.Kallisto + 1)
tmp['abs_diff'] = tmp.STAR - tmp.Kallisto

In [ ]:
# Print the total counts mapped by STAR and kallisto to the minigenome
print('STAR counts: ' + str(tmp.STAR.sum()))
print('Kallisto counts: ' + str(tmp.Kallisto.sum()))

**Note:** Kallisto does not have a whole number of counts because it uses probablistic mapping where a fraction of a read can be assigned to multiple transcripts

**Question:** How does the difference in the number of total counts between kallisto and STAR compare with the number of multimapping reads called by STAR (from above)?

**Answer:**

In [ ]:
# Plot the pearson correlation between the STAR and kallisto results
print('Pearson counts: ' + str(sp.stats.pearsonr(tmp.STAR, tmp.Kallisto)))
print('Pearson log10(counts+1): ' + str(sp.stats.pearsonr(tmp.log_STAR, tmp.log_Kallisto)))

In [ ]:
# Set the plotting size as we did last week
plt.rcParams['figure.figsize'] = [10, 10]

# Create a scatter plot comparing the log normalized counts
# s sets the size of the points, c sets the color (blue), and edgecolors sets their outline color
f = plt.figure()
scatter = plt.scatter(tmp.log_STAR, tmp.log_Kallisto, s=15, c='b', edgecolors='w')
plt.xlabel("STAR")
plt.ylabel("Kallisto")

# Label the 30 genes with the largest difference in STAR relative to kallisto using the adjust_text()
# function from the adjustText library so the names do not collide
#
# We use the sort_values() function from pandas to order the dataframe by the absolute difference
# sort_values() orders the genes in ascending order so the end contain those with the largest positive differemt
# [-30:] nabs the last 30 values
texts = []
to_plot = tmp.abs_diff.sort_values()[-30:].index
for x, y, s in zip(tmp.log_STAR[to_plot], tmp.log_Kallisto[to_plot], to_plot.to_series()):
    texts.append(plt.text(x, y, s))
    
adjust_text(texts, force_points=0.2, force_text=0.2,
            expand_points=(1, 1), expand_text=(1, 1),
            arrowprops=dict(arrowstyle="-", color='black', lw=0.5))

plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

# Create a scatter plot comparing the log normalized counts
# s sets the size of the points, c sets the color (blue), and edgecolors sets their outline color
f = plt.figure()
scatter = plt.scatter(tmp.log_STAR, tmp.log_Kallisto, s=15, c='b', edgecolors='w')
plt.xlabel("STAR")
plt.ylabel("Kallisto")

# Label the 30 genes with the largest difference in kallisto relative to STAR using the adjust_text()
# function from the adjustText library so the names do not collide
#
# We use the sort_values() function from pandas to order the dataframe by the absolute difference
# sort_values() orders the genes in ascending order so the start contains those with the largest negative differemt
# [:30] nabs the first 30 values
texts = []
to_plot = tmp.abs_diff.sort_values()[:30].index
for x, y, s in zip(tmp.log_STAR[to_plot], tmp.log_Kallisto[to_plot], to_plot.to_series()):
    texts.append(plt.text(x, y, s))

adjust_text(texts, force_points=0.2, force_text=0.2,
            expand_points=(1, 1), expand_text=(1, 1),
            arrowprops=dict(arrowstyle="-", color='black', lw=0.5))

plt.show()

In [ ]:
# Print a table of the top 30 genes by absolute difference in STAR relative to kallisto
# As sort_values() places values in ascending order we use [::-1] to reverse the dataframe
tmp.loc[tmp.abs_diff.sort_values()[-30:].index[::-1], :]

In [ ]:
# Print a table of the top 30 genes by absolute difference in kallisto relative to STAR
tmp.loc[tmp.abs_diff.sort_values()[:30].index, :]

**Note:** When finished, please save the outputs of Tutorial 2.ipynb (File > Export Notebook As... > Export Notebook to HTML) when you have completed it and upload it to Canvas.